<a href="https://colab.research.google.com/github/dmathe18/global_armed_conflict_final_repo/blob/main/2021_05_03_global_armed_conflict_analysis_danielmathew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Global Armed Conflict Analysis with Python

This notebook contains the Python code used to create the animated choropleth map of global military expenditure per capita for all countries listed in the Stockholm International Peace Research Institute (SIPRI) Military Expenditure Database, 1988-2020.

# Import Libraries

In [ ]:
# import libraries

# data analysis
import pandas as pd

# data visualization
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline

# download files to computer
from google.colab import files

# Open file from the Colab drive
import io

# Import Data from CSV files

In [ ]:
# import per capita military expenditure data from CSV
expenditure_data = "https://raw.githubusercontent.com/dmathe18/global_armed_conflict_analysis/main/military_expenditure_per_capita.csv"
df_expenditures = pd.read_csv(expenditure_data)


In [ ]:
# preview expenditure data

df_expenditures.head(10)

,Country,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Algeria,42.1,34.0,35.1,21.4,39.0,46.2,47.3,42.9,49.6,58.9,63.3,59.6,60.6,66.5,65.9,68.4,85.7,88.2,92.0,115.5,148.9,149.4,157.6,236.0,249.5,266.4,249.8,262.1,252.0,243.4,226.9,239.3,221.4
1,Libya,. .,. .,. .,. .,. .,. .,. .,. .,. .,244.6,277.8,218.6,202.6,150.6,81.8,96.3,120.0,119.2,104.4,107.0,181.6,. .,. .,. .,475.3,627.3,590.3,. .,. .,. .,. .,. .,. .
2,Morocco,38.4,40.7,43.1,45.5,47.8,47.9,51.4,53.3,52.8,50.5,51.4,42.4,29.8,50.5,50.1,61.1,64.3,66.7,69.3,77.3,93.4,95.7,97.7,102.0,102.4,120.6,118.4,94.3,94.7,97.3,102.6,102.0,130.9
3,Tunisia,29.6,29.0,30.1,30.9,33.6,31.4,33.1,37.6,42.9,38.1,38.5,37.1,34.2,34.3,35.0,41.0,44.4,46.4,48.7,47.6,55.6,53.7,53.7,66.6,62.8,69.3,82.1,87.6,87.4,75.1,73.0,85.6,97.9
4,Angola,132.8,170.1,147.8,84.2,62.7,135.7,. .,16.8,11.1,30.7,11.1,67.2,35.6,23.9,25.0,37.0,43.6,70.2,97.8,97.2,145.8,147.1,149.9,150.3,165.1,234.1,254.1,129.4,95.8,102.7,64.4,46.2,30.2
5,Benin,7.9,5.9,6.6,. .,. .,. .,. .,. .,. .,. .,. .,2.7,2.1,1.9,3.6,4.6,5.4,5.6,5.7,. .,7.4,. .,. .,. .,8.0,8.6,9.0,8.6,7.3,10.4,7.9,5.8,5.9
6,Botswana,78.1,82.6,121.4,129.9,130.5,132.7,118.8,112.9,93.4,104.2,114.9,105.3,112.4,125.6,131.1,173.6,177.9,157.8,148.7,163.4,173.5,169.0,175.5,180.3,160.3,148.6,165.8,180.9,238.2,236.9,221.0,223.4,232.1
7,Burkina Faso,6.8,7.8,9.6,7.7,7.6,6.3,3.1,3.7,3.6,3.6,3.6,3.7,3.2,3.1,3.5,4.0,5.0,5.5,5.3,7.6,8.0,8.4,7.9,8.6,8.9,9.7,10.1,8.2,8.0,10.0,15.5,17.6,18.3
8,Burundi,6.6,7.1,7.3,7.7,6.9,6.3,7.1,7.0,8.4,10.1,9.5,8.1,6.6,8.2,6.7,6.3,6.3,6.7,5.9,5.9,5.4,. .,. .,. .,6.4,6.4,6.3,6.5,6.3,5.9,5.9,7.3,5.7
9,Cameroon,13.9,12.9,15.3,14.7,14.7,13.1,7.1,8.4,8.4,8.3,9.3,9.6,7.9,7.8,8.9,11.2,12.8,12.6,14.1,15.8,18.0,17.3,17.4,16.6,16.5,17.8,17.7,15.2,16.2,16.6,17.0,16.3,14.8


In [ ]:
# import ISO ALPHA-3 country codes from CSV
iso_codes = "https://raw.githubusercontent.com/dmathe18/global_armed_conflict_analysis/main/iso_alpha_3_codes.csv"
df_codes = pd.read_csv(iso_codes)

In [ ]:
# preview country code data
df_codes.head(10)

,Country,Code
0,Afghanistan,AFG
1,Åland Islands,ALA
2,Albania,ALB
3,Algeria,DZA
4,American Samoa,ASM
5,Andorra,AND
6,Angola,AGO
7,Anguilla,AIA
8,Antigua and Barbuda,ATG
9,Argentina,ARG


# Merge and Clean Data

## Preview Data Types

In [ ]:
df_expenditures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 34 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Country  172 non-null    object
 1   1988     172 non-null    object
 2   1989     172 non-null    object
 3   1990     172 non-null    object
 4   1991     172 non-null    object
 5   1992     172 non-null    object
 6   1993     172 non-null    object
 7   1994     172 non-null    object
 8   1995     172 non-null    object
 9   1996     172 non-null    object
 10  1997     172 non-null    object
 11  1998     172 non-null    object
 12  1999     172 non-null    object
 13  2000     172 non-null    object
 14  2001     172 non-null    object
 15  2002     172 non-null    object
 16  2003     172 non-null    object
 17  2004     172 non-null    object
 18  2005     172 non-null    object
 19  2006     172 non-null    object
 20  2007     172 non-null    object
 21  2008     172 non-null    object
 22  20

In [ ]:
df_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Country  236 non-null    object
 1   Code     236 non-null    object
dtypes: object(2)
memory usage: 3.8+ KB


## Convert Objects to Floats

In [ ]:
# convert data in rows 1-34 to float
for i in df_expenditures.columns:
  if i == 'Country':
    pass
  else:
    df_expenditures[i] = pd.to_numeric(df_expenditures[i], errors = 'coerce')

df_expenditures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 34 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Country  172 non-null    object 
 1   1988     115 non-null    float64
 2   1989     116 non-null    float64
 3   1990     121 non-null    float64
 4   1991     120 non-null    float64
 5   1992     127 non-null    float64
 6   1993     139 non-null    float64
 7   1994     141 non-null    float64
 8   1995     141 non-null    float64
 9   1996     142 non-null    float64
 10  1997     144 non-null    float64
 11  1998     140 non-null    float64
 12  1999     142 non-null    float64
 13  2000     146 non-null    float64
 14  2001     147 non-null    float64
 15  2002     150 non-null    float64
 16  2003     155 non-null    float64
 17  2004     155 non-null    float64
 18  2005     157 non-null    float64
 19  2006     153 non-null    float64
 20  2007     151 non-null    float64
 21  2008     157 non

## Reconcile Spelling Variations

In [ ]:
# identify country names in expenditures dataset not found in country codes dataset
list(set(df_expenditures["Country"]) - set(df_codes["Country"]))

['North Macedonia',
 'Yugoslavia',
 'Yemen, North',
 'Czechoslovakia',
 'Czechia',
 'Côte d’Ivoire',
 'Central African Rep.',
 'Brunei',
 'Bosnia-Herzegovina',
 'Russia',
 'Trinidad & Tobago',
 'South Sudan',
 'Iran',
 'eSwatini',
 'Syria',
 'Korea, North',
 'Kosovo',
 'German DR',
 'Dominican Rep.',
 'UAE',
 'Libya',
 'USA',
 'Laos',
 'UK',
 'Taiwan',
 'Congo, Republic of ',
 'Korea, South',
 'Congo, Dem. Rep.',
 'Venezuela',
 'Tanzania']

In [ ]:
# reconcile spellings in expenditures dataset to spellings in country codes dataset
df_expenditures = df_expenditures.replace(to_replace =['Venezuela',
                                                        'Congo, Republic of ',
                                                        'Bosnia-Herzegovina',
                                                        'Congo, Dem. Rep.',
                                                        'Laos',
                                                        'eSwatini',
                                                        'Kosovo',
                                                        'Czechoslovakia',
                                                        'South Sudan',
                                                        'USA',
                                                        'UAE',
                                                        'Korea, North',
                                                        'Central African Rep.',
                                                        'Iran',
                                                        'Czechia',
                                                        'Russia',
                                                        'Dominican Rep.',
                                                        'German DR',
                                                        'Brunei',
                                                        'Libya',
                                                        'Syria',
                                                        'Korea, South',
                                                        'Tanzania',
                                                        'Yugoslavia',
                                                        'Yemen, North',
                                                        'Côte d’Ivoire',
                                                        'UK',
                                                        'Trinidad & Tobago',
                                                        'North Macedonia',
                                                        'Taiwan'],
                                              value =['Venezuela (Bolivarian Republic of)', 
                                                      'Congo', 
                                                      'Bosnia and Herzegovina',
                                                      'Democratic Republic of the Congo',
                                                      "Lao People's Democratic Republic",
                                                      'Swaziland',
                                                      'Kosovo', # not listed as an ISO standard country
                                                      'Czech Republic',
                                                      'South Sudan', # not listed as an ISO standard country
                                                      'United States of America',
                                                      'United Arab Emirates',
                                                      "Democratic People's Republic of Korea",
                                                      'Central African Republic',
                                                      'Iran, Islamic Republic of',
                                                      'Czechia', # not listed as an ISO standard country
                                                      'Russian Federation',
                                                      'Dominican Republic',
                                                      'German DR', # not listed as an ISO standard country
                                                      'Brunei Darussalam',
                                                      'Libyan Arab Jamahiriya',
                                                      'Syrian Arab Republic',
                                                      'Republic of Korea',
                                                      'United Republic of Tanzania',
                                                      'Yugoslavia', # not listed as an ISO standard country
                                                      'Yemen, North', # not listed as an ISO standard country
                                                      "Côte d'Ivoire",
                                                      'United Kingdom of Great Britain and Northern Ireland',
                                                      'Trinidad and Tobago',
                                                      'The former Yugoslav Republic of Macedonia',
                                                      'Taiwan' # not listed as an ISO standard country
                                                      ])


In [ ]:
# confirm reconciliation of country names
list(set(df_expenditures["Country"]) - set(df_codes["Country"]))

['Yugoslavia',
 'Yemen, North',
 'South Sudan',
 'Czechia',
 'Kosovo',
 'German DR',
 'Taiwan']

## Delete non-ISO Standard Countries

In [ ]:
df_expenditures = df_expenditures[df_expenditures.Country != 'Kosovo']
df_expenditures = df_expenditures[df_expenditures.Country != 'Yugoslavia']
df_expenditures = df_expenditures[df_expenditures.Country != 'Taiwan']
df_expenditures = df_expenditures[df_expenditures.Country != 'Czechia']
df_expenditures = df_expenditures[df_expenditures.Country != 'Yemen, North']
df_expenditures = df_expenditures[df_expenditures.Country != 'South Sudan']
df_expenditures = df_expenditures[df_expenditures.Country != 'German DR']


## Merge Data

In [ ]:
# merge expenditures dataframe with codes dataframe
df_merge = df_expenditures.merge(df_codes, 
                     on='Country',
                     how='left')

In [ ]:
# preview merged dataframe
df_merge.head(10)

,Country,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,Code
0,Algeria,42.1,34.0,35.1,21.4,39.0,46.2,47.3,42.9,49.6,58.9,63.3,59.6,60.6,66.5,65.9,68.4,85.7,88.2,92.0,115.5,148.9,149.4,157.6,236.0,249.5,266.4,249.8,262.1,252.0,243.4,226.9,239.3,221.4,DZA
1,Libyan Arab Jamahiriya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,244.6,277.8,218.6,202.6,150.6,81.8,96.3,120.0,119.2,104.4,107.0,181.6,NaN,NaN,NaN,475.3,627.3,590.3,NaN,NaN,NaN,NaN,NaN,NaN,LBY
2,Morocco,38.4,40.7,43.1,45.5,47.8,47.9,51.4,53.3,52.8,50.5,51.4,42.4,29.8,50.5,50.1,61.1,64.3,66.7,69.3,77.3,93.4,95.7,97.7,102.0,102.4,120.6,118.4,94.3,94.7,97.3,102.6,102.0,130.9,MAR
3,Tunisia,29.6,29.0,30.1,30.9,33.6,31.4,33.1,37.6,42.9,38.1,38.5,37.1,34.2,34.3,35.0,41.0,44.4,46.4,48.7,47.6,55.6,53.7,53.7,66.6,62.8,69.3,82.1,87.6,87.4,75.1,73.0,85.6,97.9,TUN
4,Angola,132.8,170.1,147.8,84.2,62.7,135.7,NaN,16.8,11.1,30.7,11.1,67.2,35.6,23.9,25.0,37.0,43.6,70.2,97.8,97.2,145.8,147.1,149.9,150.3,165.1,234.1,254.1,129.4,95.8,102.7,64.4,46.2,30.2,AGO
5,Benin,7.9,5.9,6.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.7,2.1,1.9,3.6,4.6,5.4,5.6,5.7,NaN,7.4,NaN,NaN,NaN,8.0,8.6,9.0,8.6,7.3,10.4,7.9,5.8,5.9,BEN
6,Botswana,78.1,82.6,121.4,129.9,130.5,132.7,118.8,112.9,93.4,104.2,114.9,105.3,112.4,125.6,131.1,173.6,177.9,157.8,148.7,163.4,173.5,169.0,175.5,180.3,160.3,148.6,165.8,180.9,238.2,236.9,221.0,223.4,232.1,BWA
7,Burkina Faso,6.8,7.8,9.6,7.7,7.6,6.3,3.1,3.7,3.6,3.6,3.6,3.7,3.2,3.1,3.5,4.0,5.0,5.5,5.3,7.6,8.0,8.4,7.9,8.6,8.9,9.7,10.1,8.2,8.0,10.0,15.5,17.6,18.3,BFA
8,Burundi,6.6,7.1,7.3,7.7,6.9,6.3,7.1,7.0,8.4,10.1,9.5,8.1,6.6,8.2,6.7,6.3,6.3,6.7,5.9,5.9,5.4,NaN,NaN,NaN,6.4,6.4,6.3,6.5,6.3,5.9,5.9,7.3,5.7,BDI
9,Cameroon,13.9,12.9,15.3,14.7,14.7,13.1,7.1,8.4,8.4,8.3,9.3,9.6,7.9,7.8,8.9,11.2,12.8,12.6,14.1,15.8,18.0,17.3,17.4,16.6,16.5,17.8,17.7,15.2,16.2,16.6,17.0,16.3,14.8,CMR


## Transform Data (For Animated Choropleth Map)

In [ ]:
# create list of years in merged dataset
years = df_merge.columns[1:-1].tolist()
print(years)

['1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']


In [ ]:
# create list of countries in merged dataset
countries = df_merge['Country'].unique().tolist()
print(countries)

['Algeria', 'Libyan Arab Jamahiriya', 'Morocco', 'Tunisia', 'Angola', 'Benin', 'Botswana', 'Burkina Faso', 'Burundi', 'Cameroon', 'Cape Verde', 'Central African Republic', 'Chad', 'Congo', 'Democratic Republic of the Congo', "Côte d'Ivoire", 'Djibouti', 'Equatorial Guinea', 'Eritrea', 'Ethiopia', 'Gabon', 'Gambia', 'Ghana', 'Guinea', 'Guinea-Bissau', 'Kenya', 'Lesotho', 'Liberia', 'Madagascar', 'Malawi', 'Mali', 'Mauritania', 'Mauritius', 'Mozambique', 'Namibia', 'Niger', 'Nigeria', 'Rwanda', 'Senegal', 'Seychelles', 'Sierra Leone', 'Somalia', 'South Africa', 'Sudan', 'Swaziland', 'United Republic of Tanzania', 'Togo', 'Uganda', 'Zambia', 'Zimbabwe', 'Belize', 'Costa Rica', 'Cuba', 'Dominican Republic', 'El Salvador', 'Guatemala', 'Haiti', 'Honduras', 'Jamaica', 'Mexico', 'Nicaragua', 'Panama', 'Trinidad and Tobago', 'Canada', 'United States of America', 'Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Guyana', 'Paraguay', 'Peru', 'Uruguay', 'Venezuela (Bolivarian Rep

In [ ]:
# create list of ISO alpha-3 country codes
codes = df_merge['Code'].unique().tolist()
print(codes)


['DZA', 'LBY', 'MAR', 'TUN', 'AGO', 'BEN', 'BWA', 'BFA', 'BDI', 'CMR', 'CPV', 'CAF', 'TCD', 'COG', 'COD', 'CIV', 'DJI', 'GNQ', 'ERI', 'ETH', 'GAB', 'GMB', 'GHA', 'GIN', 'GNB', 'KEN', 'LSO', 'LBR', 'MDG', 'MWI', 'MLI', 'MRT', 'MUS', 'MOZ', 'NAM', 'NER', 'NGA', 'RWA', 'SEN', 'SYC', 'SLE', 'SOM', 'ZAF', 'SDN', 'SWZ', 'TZA', 'TGO', 'UGA', 'ZMB', 'ZWE', 'BLZ', 'CRI', 'CUB', 'DOM', 'SLV', 'GTM', 'HTI', 'HND', 'JAM', 'MEX', 'NIC', 'PAN', 'TTO', 'CAN', 'USA', 'ARG', 'BOL', 'BRA', 'CHL', 'COL', 'ECU', 'GUY', 'PRY', 'PER', 'URY', 'VEN', 'KAZ', 'KGZ', 'TJK', 'TKM', 'UZB', 'CHN', 'JPN', 'PRK', 'KOR', 'MNG', 'AFG', 'BGD', 'IND', 'NPL', 'PAK', 'LKA', 'BRN', 'KHM', 'IDN', 'LAO', 'MYS', 'MMR', 'PHL', 'SGP', 'THA', 'TLS', 'VNM', 'AUS', 'FJI', 'NZL', 'PNG', 'ALB', 'BIH', 'BGR', 'HRV', 'CZE', 'EST', 'HUN', 'LVA', 'LTU', 'MNE', 'MKD', 'POL', 'ROU', 'SRB', 'SVK', 'SVN', 'ARM', 'AZE', 'BLR', 'GEO', 'MDA', 'RUS', 'UKR', 'AUT', 'BEL', 'CYP', 'DNK', 'FIN', 'FRA', 'DEU', 'GRC', 'ISL', 'IRL', 'ITA', 'LUX', 'MLT'

In [ ]:
# initialize two-dimensional list
df_expanded = []
# initialize temporary list
temp = []

# use for loop to create two dimensional list with columns country, year, expenditure, and code
for i in range(len(countries)):
  for j in range(len(years)):
    temp.append(countries[i])
    temp.append(years[j])
    temp.append(df_merge[years[j]][i])
    temp.append(codes[i])
    df_expanded.append(temp)
    temp=[]

# preview two-dimensional list
df_expanded[:100]

[['Algeria', '1988', 42.1, 'DZA'],
 ['Algeria', '1989', 34.0, 'DZA'],
 ['Algeria', '1990', 35.1, 'DZA'],
 ['Algeria', '1991', 21.4, 'DZA'],
 ['Algeria', '1992', 39.0, 'DZA'],
 ['Algeria', '1993', 46.2, 'DZA'],
 ['Algeria', '1994', 47.3, 'DZA'],
 ['Algeria', '1995', 42.9, 'DZA'],
 ['Algeria', '1996', 49.6, 'DZA'],
 ['Algeria', '1997', 58.9, 'DZA'],
 ['Algeria', '1998', 63.3, 'DZA'],
 ['Algeria', '1999', 59.6, 'DZA'],
 ['Algeria', '2000', 60.6, 'DZA'],
 ['Algeria', '2001', 66.5, 'DZA'],
 ['Algeria', '2002', 65.9, 'DZA'],
 ['Algeria', '2003', 68.4, 'DZA'],
 ['Algeria', '2004', 85.7, 'DZA'],
 ['Algeria', '2005', 88.2, 'DZA'],
 ['Algeria', '2006', 92.0, 'DZA'],
 ['Algeria', '2007', 115.5, 'DZA'],
 ['Algeria', '2008', 148.9, 'DZA'],
 ['Algeria', '2009', 149.4, 'DZA'],
 ['Algeria', '2010', 157.6, 'DZA'],
 ['Algeria', '2011', 236.0, 'DZA'],
 ['Algeria', '2012', 249.5, 'DZA'],
 ['Algeria', '2013', 266.4, 'DZA'],
 ['Algeria', '2014', 249.8, 'DZA'],
 ['Algeria', '2015', 262.1, 'DZA'],
 ['Algeria'

In [ ]:
# convert two-dimensional list to pandas DataFrame
df_transformed = pd.DataFrame(df_expanded, columns =['Country', 'Year', 'Expenditure', 'Code'])

# preview DataFrame
df_transformed.head(10)


,Country,Year,Expenditure,Code
0,Algeria,1988,42.1,DZA
1,Algeria,1989,34.0,DZA
2,Algeria,1990,35.1,DZA
3,Algeria,1991,21.4,DZA
4,Algeria,1992,39.0,DZA
5,Algeria,1993,46.2,DZA
6,Algeria,1994,47.3,DZA
7,Algeria,1995,42.9,DZA
8,Algeria,1996,49.6,DZA
9,Algeria,1997,58.9,DZA


# Create Visualizations

## 1988 Global Military Expenditure Map

In [ ]:
df_1988 = df_transformed[df_transformed['Year'] == '1988']


df_1988.head(10)

,Country,Year,Expenditure,Code
0,Algeria,1988,42.1,DZA
33,Libyan Arab Jamahiriya,1988,NaN,LBY
66,Morocco,1988,38.4,MAR
99,Tunisia,1988,29.6,TUN
132,Angola,1988,132.8,AGO
165,Benin,1988,7.9,BEN
198,Botswana,1988,78.1,BWA
231,Burkina Faso,1988,6.8,BFA
264,Burundi,1988,6.6,BDI
297,Cameroon,1988,13.9,CMR


In [ ]:
choropleth_expenditures_1988 = px.choropleth(df_1988,
                                             locationmode='ISO-3',
                                             locations='Code',
                                             color='Expenditure',
                                             color_continuous_scale='plasma',
                                             labels={'Expenditure':'Expenditure (Current US$)', 'Code':'Country Code', 'Year':'Calendar Year'},
                                             title = 'Global Military Expenditure Per Capita, 1988'
                                             )

In [ ]:
choropleth_expenditures_1988

## Animated Choropleth Map

In [ ]:
animated_choropleth_expenditures = px.choropleth(df_transformed,
                                             locationmode='ISO-3',
                                             locations='Code',
                                             color='Expenditure',
                                             color_continuous_scale='plasma',
                                             labels={'Expenditure':'Expenditure (Current US$)', 'Code':'Country Code', 'Year':'Calendar Year'},
                                             title = 'Military Expenditure Per Capita, 1988-2020',
                                             animation_frame='Year'
                                             )

In [ ]:
animated_choropleth_expenditures

# Export to HTML

In [ ]:
# save as html to share animation
animated_choropleth_expenditures.write_html('animated_choropleth_expenditures.html')
choropleth_expenditures_1988.write_html('choropleth_expenditures_1988.html')

In [ ]:
# download the files from Google Colab
files.download('animated_choropleth_expenditures.html')
files.download('choropleth_expenditures_1988.html')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>